Diagnostics for NR

In [1]:
import sys
sys.path.append("../")

import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

import gw_eccentricity
from gw_eccentricity import get_available_methods
from gw_eccentricity import measure_eccentricity
from gw_eccentricity.load_data import load_waveform
from gw_eccentricity.plot_settings import colorsDict, lstyles, lwidths

%load_ext autoreload
%autoreload 2

/home/arif/miniconda3/envs/ecc/lib/python3.9/site-packages/gwtools/rotations.py:63: UserWarning: Could not import GWFrames, needed for rotations module
  _warnings.warn("Could not import GWFrames, needed for rotations module")
/home/arif/miniconda3/envs/ecc/lib/python3.9/site-packages/gwtools/__init__.py:11: UserWarning: Could not import rotations, decompositions, or fitfuncs. These are not needed by GWSurrogate.
  _warnings.warn("Could not import rotations, decompositions, or fitfuncs. These are not needed by GWSurrogate.")


In [2]:
import glob
import numpy as np
from tqdm import tqdm
from gw_eccentricity.load_data import load_lvcnr_hack

In [3]:
sxs_nr_waveforms = sorted(glob.glob("../data/ecc_waveforms/Non-Precessing/SXS/BBH_S*.h5"))
et_waveforms = sorted(glob.glob("../data/ecc_waveforms/Non-Precessing/ET/*.h5"))

In [4]:
all_nr_waveforms = sorted(np.append(sxs_nr_waveforms, et_waveforms))

In [5]:
# in sxs_nr_waveforms
problematic_cases = [8, # need to increase flow and width 
                     34, # Takes too long to load, did not load within tested time.
                     37, # loads too short waveform
                    ]

In [6]:
num_orbits_dict= {"BBH_SKS_q1_e09_D130_DoNotControlLEqualsOneShape_AdjustSphereCExtents_Res3.h5": 3,
                 "BBH_SKS_q1_e09_D130_SKS_DoNotControlLEqualsOneShape_Res3.h5": 3}

In [7]:
def MakeDiagnosticPlots(methods, nr_waveforms, num_orbits_dict):
    """loop through all NR waveforms, and make diagnostics plots 
         methods - list of methods to apply
         nr_waveforms - list of paths to NR waveforms to apply methods to
    """

    # make output objects, one per method
    pps = [PdfPages(f"NR-diagnostics-{method}.pdf") for method in methods]


    comparisons = PdfPages("NR_diagnostics-comparisons.pdf")

    for idx, waveform_path in enumerate(nr_waveforms):
        w = waveform_path.split("/")[-1]
        print(f"===== {idx}/{len(nr_waveforms)} ===== {w} =====", flush=True)
        kwargs = {"filepath": waveform_path,
                "include_zero_ecc": True,
                "num_orbits_to_remove_as_junk": num_orbits_dict.get(w, 3)}
        dataDict = load_lvcnr_hack(**kwargs)
        tref_vec = dataDict["t"]


        # for ecc-comparison plot
        fig0, ax0 = plt.subplots(figsize=(12, 6))

        for method,pp in zip(methods, pps):
            tref_out, ecc_vec, mean_ano_vec, eccMethod = measure_eccentricity(
                tref_in=tref_vec,
                dataDict=dataDict,
                method=method, 
                return_gwecc_object=True,
                extra_kwargs={"debug": False})
            fig, ax = eccMethod.make_diagnostic_plots(twocol=True)
            q = dataDict["params_dict"]["q"]
            ecc = dataDict["params_dict"]["ecc"]
            chi1 = np.around(dataDict["params_dict"]["chi1"], 5)
            chi2 = np.around(dataDict["params_dict"]["chi2"], 5)
            title = (f"case {idx}/{len(sxs_nr_waveforms)-1}, "
                    f"$q$={q:.1f}, "
                    f"$\chi_{1}$={chi1}, "
                    f"$\chi_{2}$={chi2}, "
                    f"ecc={ecc:.2f}\n"
                    f"method={method}, "
                    f"id={w}")
            fig.suptitle(title, ha="center", fontsize=10)
            fig.set_tight_layout(True)
            fig.savefig(pp,format='pdf')
            plt.close(fig)

            eccMethod.plot_measured_ecc(fig0, ax0, add_vline_at_tref=True if method == methods[0] else False, **{"label": method, "lw": lwidths[method], "ls": lstyles[method], "c": colorsDict[method]})
        ax0.set_title(f"q={q:.1f}, ecc={ecc:.2f}, {w}", fontsize=14)
        ax0.legend();
        fig0.savefig(comparisons,format='pdf')
        plt.close(fig0)

    for pp in pps:
        pp.close()
    comparisons.close()

In [8]:
# to test, analyse only two waveforms:
MakeDiagnosticPlots(gw_eccentricity.get_available_methods(), sxs_nr_waveforms[30:32], num_orbits_dict)

===== 0/2 ===== BBH_SHK_q3_e04_D28_Res3.h5 =====
===== 1/2 ===== BBH_SHK_q3_e08_D55_Res3.h5 =====


In [ ]:
# ALL SXS-WAVEFORMS.  THIS TAKES A FEW MINUTES !!!
MakeDiagnosticPlots(gw_eccentricity.get_available_methods(), sxs_nr_waveforms, num_orbits_dict)

===== 0/61 ===== BBH_SHK_q10_e01_D14.5_Res3.h5 =====


../gw_eccentricity/eccDefinition.py:450: UserWarning: Encountered negative eccentricity.
  warnings.warn("Encountered negative eccentricity.")


===== 1/61 ===== BBH_SHK_q10_e01_D16_Res3.h5 =====
===== 2/61 ===== BBH_SHK_q10_e05_D28_Res3.h5 =====
===== 3/61 ===== BBH_SHK_q10_e05_D30_Res3.h5 =====
===== 4/61 ===== BBH_SHK_q10_e07_D45_Res3.h5 =====
===== 5/61 ===== BBH_SHK_q1_0_0_e01_D16_FastDyn_Res3.h5 =====
===== 6/61 ===== BBH_SHK_q1_0_0_e01_D20_Res3.h5 =====
===== 7/61 ===== BBH_SHK_q1_0_0_e02_D18_Res3.h5 =====
===== 8/61 ===== BBH_SHK_q1_0_0_e02_D22_Res3.h5 =====
===== 9/61 ===== BBH_SHK_q1_0_0_e05_D34_Res3.h5 =====
===== 10/61 ===== BBH_SHK_q1_e035_D26_Res3.h5 =====
===== 11/61 ===== BBH_SHK_q1_e04_D28_Res3.h5 =====
===== 12/61 ===== BBH_SHK_q1_e07_D45_Res3.h5 =====
===== 13/61 ===== BBH_SHK_q1_e08_D60_Res3.h5 =====
===== 14/61 ===== BBH_SHK_q1_e095_D65_Res3.h5 =====
===== 15/61 ===== BBH_SHK_q1_e09_D70_Res3.h5 =====
===== 16/61 ===== BBH_SHK_q2_e01_D18_Res3.h5 =====
===== 17/61 ===== BBH_SHK_q2_e02_D20_Res3.h5 =====
===== 18/61 ===== BBH_SHK_q2_e035_D26_Res3.h5 =====
===== 19/61 ===== BBH_SHK_q2_e04_D28_Res3.h5 =====
=====